In [1]:
#Import Dependencies
import pandas as pd
import numpy as np

from sqlalchemy import create_engine
import psycopg2
import sqlalchemy
from sqlalchemy.ext.automap import automap_base
from sqlalchemy.orm import Session
from sqlalchemy import create_engine, func, inspect

In [2]:
url2020 = 'http://archive.shanghairanking.com/ARWU2020.html'
url2019 = 'http://archive.shanghairanking.com/ARWU2019.html'
url2018 = 'http://archive.shanghairanking.com/ARWU2018.html'
url2017 = 'http://archive.shanghairanking.com/ARWU2017.html'
url2016 = 'http://archive.shanghairanking.com/ARWU2016.html'

In [3]:
rank2020 = pd.read_html(url2020)
rank2019 = pd.read_html(url2019)
rank2018 = pd.read_html(url2018)
rank2017 = pd.read_html(url2017)
rank2016 = pd.read_html(url2016)

In [4]:
rank2020 = rank2020[0]
rank2019 = rank2019[0]
rank2018 = rank2018[0]
rank2017 = rank2017[0]
rank2016 = rank2016[0]

In [5]:
rank2020['year'] = 2020
rank2019['year'] = 2019
rank2018['year'] = 2018
rank2017['year'] = 2017
rank2016['year'] = 2016

In [6]:
rank = rank2020.append(rank2019)
rank = rank.append(rank2018)
rank = rank.append(rank2017)
rank = rank.append(rank2016)
rank = rank[['Institution*','World Rank','National/Regional Rank','year']]
rank = rank.reset_index(drop=True)
rank

,Institution*,World Rank,National/Regional Rank,year
0,Harvard University,1,1,2020
1,Stanford University,2,2,2020
2,University of Cambridge,3,1,2020
3,Massachusetts Institute of Technology (MIT),4,3,2020
4,"University of California, Berkeley",5,4,2020
...,...,...,...,...
3495,Wake Forest University,401-500,120-137,2016
3496,Waseda University,401-500,13-16,2016
3497,Washington State University,401-500,120-137,2016
3498,West Virginia University,401-500,120-137,2016


In [7]:
unique_u = rank.drop_duplicates(subset = 'Institution*', ignore_index=True)

In [8]:
unique_u = pd.DataFrame(unique_u['Institution*'])
unique_u

,Institution*
0,Harvard University
1,Stanford University
2,University of Cambridge
3,Massachusetts Institute of Technology (MIT)
4,"University of California, Berkeley"
...,...
1106,Toulouse School of Economics
1107,University of Quebec
1108,"University of Technology, Sydney"
1109,National Cheng Kung University (NCKU)


In [9]:
us_inst = pd.read_csv('../resources/nces/delta_public_00_12.csv', low_memory=False)
us_inst = us_inst.drop_duplicates(subset='instname')

In [10]:
us = pd.DataFrame()
us['Institution*'] = us_inst['instname']
us

,Institution*
0,Brown College-Brooklyn Center
1,Marian Health Careers Center-Van Nuys Campus
2,Springfield College-School of Human Services
3,American Broadcasting School-Online Program
4,Kaplan College-Chesapeake
...,...
11592,DeVry University-Oklahoma
11593,DeVry University-Utah
11594,Devry University-Michigan
11595,DeVry University-Tennessee


In [11]:
world_u = unique_u.append(us, ignore_index=True)
world_u = world_u.rename(columns={'Institution*': 'institution'})
world_u = world_u.sort_values(by=['institution'], ignore_index=True)
world_u

,institution
0,360 Degrees Beauty Academy
1,4-States Academy of Cosmetology
2,4-States Okmulgee Academy of Cosmetology
3,5 STAR TRAVEL CAREERS
4,A & W Healthcare Educators
...,...
12703,Zhejiang University
12704,Zhejiang University of Technology
12705,Zhengzhou University
12706,Zion Bible College


In [12]:
world_u.loc[world_u['institution'] == 'Harvard University']

,institution
4691,Harvard University
4692,Harvard University


In [13]:
world_u = world_u.drop_duplicates(subset='institution')
world_u = world_u.reset_index(drop=True, inplace=False)

In [14]:
world_u.loc[world_u['institution'] == 'Harvard University']

,institution
4659,Harvard University


In [15]:
world_u.to_csv('../resources/world_universities.csv')

In [16]:
rk10 = pd.read_csv('../resources/world_universities.csv')
rk10 = rk10.rename(columns={'Unnamed: 0':'i_id'})
rk10 = rk10.merge(rank, left_on='institution', right_on='Institution*', how='inner')
rk10

,i_id,institution,Institution*,World Rank,National/Regional Rank,year
0,88,AGH University of Science and Technology,AGH University of Science and Technology,701-800,3,2020
1,88,AGH University of Science and Technology,AGH University of Science and Technology,601-700,3,2019
2,318,AUT University,AUT University,901-1000,8,2020
3,318,AUT University,AUT University,701-800,6-7,2019
4,332,Aalborg University,Aalborg University,301-400,4-5,2020
...,...,...,...,...,...,...
3495,12571,Zhejiang University,Zhejiang University,101-150,3-6,2016
3496,12572,Zhejiang University of Technology,Zhejiang University of Technology,501-600,72-84,2020
3497,12572,Zhejiang University of Technology,Zhejiang University of Technology,501-600,59-72,2019
3498,12573,Zhengzhou University,Zhengzhou University,301-400,33-49,2020


In [17]:
rk10 = rk10.drop({'institution','Institution*'}, axis=1)
rk10 = rk10.rename(columns={'World Rank':'wranking', 'National/Regional Rank':'nranking'})
rk10

,i_id,wranking,nranking,year
0,88,701-800,3,2020
1,88,601-700,3,2019
2,318,901-1000,8,2020
3,318,701-800,6-7,2019
4,332,301-400,4-5,2020
...,...,...,...,...
3495,12571,101-150,3-6,2016
3496,12572,501-600,72-84,2020
3497,12572,501-600,59-72,2019
3498,12573,301-400,33-49,2020


In [18]:
# Clean range on rankings to get a numeral value
wr = rk10['wranking']
nr = rk10['nranking']

a = wr.str.split("-", expand=True)
b = nr.str.split("-", expand=True)

rk10['wrank2'] = a[0]
rk10['nrank2'] = b[0]

rk10

,i_id,wranking,nranking,year,wrank2,nrank2
0,88,701-800,3,2020,701,3
1,88,601-700,3,2019,601,3
2,318,901-1000,8,2020,901,8
3,318,701-800,6-7,2019,701,6
4,332,301-400,4-5,2020,301,4
...,...,...,...,...,...,...
3495,12571,101-150,3-6,2016,101,3
3496,12572,501-600,72-84,2020,501,72
3497,12572,501-600,59-72,2019,501,59
3498,12573,301-400,33-49,2020,301,33


In [19]:
rk10['wranking'] = rk10['wrank2']
rk10['nranking'] = rk10['nrank2']
rk10

,i_id,wranking,nranking,year,wrank2,nrank2
0,88,701,3,2020,701,3
1,88,601,3,2019,601,3
2,318,901,8,2020,901,8
3,318,701,6,2019,701,6
4,332,301,4,2020,301,4
...,...,...,...,...,...,...
3495,12571,101,3,2016,101,3
3496,12572,501,72,2020,501,72
3497,12572,501,59,2019,501,59
3498,12573,301,33,2020,301,33


In [20]:
rk10 = rk10.drop(columns={'wrank2', 'nrank2'})
rk10

,i_id,wranking,nranking,year
0,88,701,3,2020
1,88,601,3,2019
2,318,901,8,2020
3,318,701,6,2019
4,332,301,4,2020
...,...,...,...,...
3495,12571,101,3,2016
3496,12572,501,72,2020
3497,12572,501,59,2019
3498,12573,301,33,2020


In [21]:
rk10.to_csv('../resources/ARWA2016-2020.csv', index=False)

In [22]:
us1 = pd.read_csv('../resources/world_universities.csv')
us1 = us1.rename(columns={'Unnamed: 0':'i_id'})
us1 = us1.merge(us_inst, left_on='institution', right_on='instname', how='inner')
us1

,i_id,institution,instname,city,state,zip
0,0,360 Degrees Beauty Academy,360 Degrees Beauty Academy,Houston,TX,77084
1,1,4-States Academy of Cosmetology,4-States Academy of Cosmetology,Vinita,OK,74301
2,2,4-States Okmulgee Academy of Cosmetology,4-States Okmulgee Academy of Cosmetology,Okmulgee,OK,74447
3,3,5 STAR TRAVEL CAREERS,5 STAR TRAVEL CAREERS,WHITE OAK,PA,15131
4,4,A & W Healthcare Educators,A & W Healthcare Educators,New Orleans,LA,70122
...,...,...,...,...,...,...
11592,12566,Zarem Golde ORT Technical Institute,Zarem Golde ORT Technical Institute,Skokie,IL,60077
11593,12567,Zarem-Golde ORT Technical Institute,Zarem-Golde ORT Technical Institute,Skokie,IL,60077
11594,12568,Zaytuna College,Zaytuna College,Berkeley,CA,94704
11595,12574,Zion Bible College,Zion Bible College,Barrington,RI,2806


In [23]:
us1 = us1.drop({'institution','instname'}, axis=1)
us1

,i_id,city,state,zip
0,0,Houston,TX,77084
1,1,Vinita,OK,74301
2,2,Okmulgee,OK,74447
3,3,WHITE OAK,PA,15131
4,4,New Orleans,LA,70122
...,...,...,...,...
11592,12566,Skokie,IL,60077
11593,12567,Skokie,IL,60077
11594,12568,Berkeley,CA,94704
11595,12574,Barrington,RI,2806


In [24]:
# Clean null values
us1 = us1.dropna()
us1

,i_id,city,state,zip
0,0,Houston,TX,77084
1,1,Vinita,OK,74301
2,2,Okmulgee,OK,74447
3,3,WHITE OAK,PA,15131
4,4,New Orleans,LA,70122
...,...,...,...,...
11592,12566,Skokie,IL,60077
11593,12567,Skokie,IL,60077
11594,12568,Berkeley,CA,94704
11595,12574,Barrington,RI,2806


In [25]:
# Save to csv to be load in the database
us1.to_csv('../resources/us_colleges.csv', index=False)